In [1]:
import sys
sys.path.append("../")

import argparse
import datetime
import json
import os
import re
from time import sleep


from sshtunnel import SSHTunnelForwarder
import pymysql

import numpy as np
import pandas as pd
import requests

from vook_db_v7.config import MAX_PAGE, REQ_URL, WANT_ITEMS, req_params
from vook_db_v7.local_config import *

# from logzero import logger

In [2]:
"""DBからテーブル取得"""

config_ec2 = get_ec2_config()
query = """
SELECT
    *
FROM
    products 
"""
df_from_db = pd.DataFrame()

# SSHトンネルの設定
with SSHTunnelForwarder(
    (config_ec2["host_name"], config_ec2["ec2_port"]),
    ssh_username=config_ec2["ssh_username"],
    ssh_pkey=config_ec2["ssh_pkey"],
    remote_bind_address=(
        config_ec2["rds_end_point"],
        config_ec2["rds_port"],
    ),
) as server:
    print(f"Local bind port: {server.local_bind_port}")
    conn = None

    try:
        conn = pymysql.connect(
            **get_rds_config(server.local_bind_port), connect_timeout=10
        )
        cursor = conn.cursor()
        # SQLクエリの実行
        cursor.execute(query)
        for row in cursor:  # column1, column2, ...は取得したいカラム名に合わせて変更してください
            #print(row)
            
            df_from_db = pd.concat([df_from_db, pd.DataFrame([row])], ignore_index=True)

    except pymysql.MySQLError as e:
        print(f"Error connecting to MySQL: {e}")
    finally:
        if conn is not None:
            conn.close()

Local bind port: 51118


In [3]:
df_from_db
# 新しいデータ
new_data = {'id': 9999999, 'name': 'test', 'url': 'test', 'price': 9999999, 'knowledge_id': 88, 'platform_id': 1, 'size_id': 999,
           'created_at': '2023-11-19 13:14:08.081758', 'updated_at': '2023-11-19 13:14:08.081758'}
df_new = pd.concat([df_from_db, pd.DataFrame([new_data])], ignore_index=True)
df_new.tail()

,id,name,url,price,knowledge_id,platform_id,size_id,created_at,updated_at
2702,3056,【中古】Lee◆カバーオール/34/ナイロン/カーキ/91-J【メンズウェア】,https://hb.afl.rakuten.co.jp/hgc/g00r3cee.brn1...,4290,88,1,999,2023-11-19 13:14:08.081758,2023-11-19 13:14:08.081758
2703,3057,【中古】 Brenda Lee / Here Comes That Feeling 輸入盤 ...,https://hb.afl.rakuten.co.jp/hgc/g00qs41e.brn1...,440,88,1,999,2023-11-19 13:14:08.081758,2023-11-19 13:14:08.081758
2704,3058,【中古】 Brenda Lee / Here Comes That Feeling 輸入盤 ...,https://hb.afl.rakuten.co.jp/hgc/g00svqpe.brn1...,440,88,1,999,2023-11-19 13:14:08.081758,2023-11-19 13:14:08.081758
2705,3059,【中古】 Brenda Lee / Here Comes That Feeling 輸入盤 ...,https://hb.afl.rakuten.co.jp/hgc/g00tbz7e.brn1...,390,88,1,999,2023-11-19 13:14:08.081758,2023-11-19 13:14:08.081758
2706,9999999,test,test,9999999,88,1,999,2023-11-19 13:14:08.081758,2023-11-19 13:14:08.081758


In [4]:
def get_rds_config_for_put(port):
    return {
        "user": "root",
        "password": "rds-vook",
        "port": port,
        "host": "localhost",
        "database": "vook_web_v3_development",
        "charset": "utf8mb4",
        "cursorclass": pymysql.cursors.DictCursor,
    }

In [5]:
'id','name','url','price','knowledge_id','platform_id','size_id','created_at','updated_at'

('id',
 'name',
 'url',
 'price',
 'knowledge_id',
 'platform_id',
 'size_id',
 'created_at',
 'updated_at')

In [6]:
config_ec2 = put_ec2_config()

# SSHトンネルの設定
with SSHTunnelForwarder(
    (config_ec2["host_name"], config_ec2["ec2_port"]),
    ssh_username=config_ec2["ssh_username"],
    ssh_pkey=config_ec2["ssh_pkey"],
    remote_bind_address=(
        config_ec2["rds_end_point"],
        config_ec2["rds_port"],
    ),
) as server:
    print(f"Local bind port: {server.local_bind_port}")
    conn = None

    try:
        conn = pymysql.connect(
            **get_rds_config_for_put(server.local_bind_port), connect_timeout=10
        )
        cursor = conn.cursor()
        # SQLクエリの実行
        print("ここに処理を書く")
        
        create_table_query = """
                CREATE TABLE IF NOT EXISTS products (
                    id bigint PRIMARY KEY AUTO_INCREMENT,
                    name varchar(255) NOT NULL,
                    url varchar(255) NOT NULL UNIQUE,
                    price int NOT NULL,
                    knowledge_id bigint NOT NULL,
                    platform_id bigint NOT NULL,
                    size_id bigint NOT NULL,
                    created_at datetime(6) NOT NULL,
                    updated_at datetime(6) NOT NULL
                )
            """
        cursor.execute(create_table_query)
        # DataFrameをRDSのテーブルに挿入
        insert_query = "INSERT INTO products (id,name,url,price,knowledge_id,platform_id,size_id,created_at,updated_at) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    
        for index, row in df_new.iterrows():
            print(row)
            cursor.execute(insert_query, (row['id'], row['name'], row['url'],row['price'], row['knowledge_id'], row['platform_id'],
                                         row['size_id'], row['created_at'], row['updated_at']))
        conn.commit()

    except pymysql.MySQLError as e:
        print(f"Error connecting to MySQL: {e}")
    finally:
        if conn is not None:
            conn.close()

Local bind port: 51122
ここに処理を書く
id                                                              1
name            Levi's リーバイス デニムパンツ ヴィンテージ 501 66前期 ビックE トップボタ...
url             https://hb.afl.rakuten.co.jp/hgc/g00rc6oe.brn1...
price                                                      418979
knowledge_id                                                    5
platform_id                                                     1
size_id                                                       999
created_at                             2023-09-30 12:30:42.414466
updated_at                             2023-09-30 12:30:42.414466
Name: 0, dtype: object
Error connecting to MySQL: (1062, "Duplicate entry '1' for key 'products.PRIMARY'")
